# initial clean data of Bicicletar datasets
## to-do list
 - load the datasets
 - rename column names
 - combine datetime columns
 - remove columns with duplication information
 - remove rows with jornadas < t_limit

In [1]:
import pandas as pd
import numpy as np
import time
import datetime

### 2015

In [2]:
# load the data
brute_path_2015 = '../data/brute/2015.xlsx'
brute_data_2015 = pd.read_excel(brute_path_2015)

# change the columns name
clean_data_2015 = brute_data_2015.rename(index=str, columns={'IdJornada': 'id_jornada',
                                                             'IdUsuario': 'id_usuario',
                                                             'AnoNascimento': 'ano_nascimento',
                                                             'Sexo': 'sexo',
                                                             'Pais': 'pais',
                                                             'Distrito': 'distrito',
                                                             'Cidade': 'cidade',
                                                             'UF': 'uf',
                                                             'DataCadastro': 'data_cadastro',
                                                             'Meio de contato para retirada': 'tipo_usuario',
                                                             'Bicicleta': 'id_bicicleta',
                                                             'DataRetirada': 'data_retirada',
                                                             'HoraRetirada': 'hora_retirada',
                                                             'DataDevolucao': 'data_devolucao',
                                                             'HoraDevolucao': 'hora_devolucao',
                                                             'EstacaoRetirada': 'estacao_retirada',
                                                             'EstacaoDevolucao': 'estacao_devolucao'});
# set id_jornada to index
clean_data_2015 = clean_data_2015.set_index('id_jornada')

# change type of data_cadastro
clean_data_2015['data_cadastro'] = clean_data_2015['data_cadastro'].astype('datetime64')

# combine data_retirada with hora_retirada
clean_data_2015['datetime_retirada'] = clean_data_2015['data_retirada'] + pd.to_timedelta(clean_data_2015['hora_retirada'])

# combine data_devolucao with hora_devolucao
clean_data_2015['datetime_devolucao'] = clean_data_2015['data_devolucao'] + pd.to_timedelta(clean_data_2015['hora_devolucao'])

# drop columns with duplicate information columns
clean_data_2015 = clean_data_2015.drop(['TempoJornada',
                                        'TempoJornadaMinutos',
                                        'data_retirada',
                                        'hora_retirada',
                                        'data_devolucao',
                                        'hora_devolucao',
                                        'distrito'], axis=1)

### 2016

In [ ]:
# load the data
brute_path_2016 = '../data/brute/2016.xlsx'
brute_data_2016 = pd.read_excel(brute_path_2016)

# change the columns name
clean_data_2016 = brute_data_2016.rename(index=str, columns={'IdJornada': 'id_jornada',
                                                             'IdUsuario': 'id_usuario',
                                                             'AnoNascimento': 'ano_nascimento',
                                                             'Sexo': 'sexo',
                                                             'Pais': 'pais',
                                                             'Distrito': 'distrito',
                                                             'Cidade': 'cidade',
                                                             'UF': 'uf',
                                                             'DataCadastro': 'data_cadastro',
                                                             'Meio de contato para retirada': 'tipo_usuario',
                                                             'Bicicleta': 'id_bicicleta',
                                                             'DataRetirada': 'data_retirada',
                                                             'HoraRetirada': 'hora_retirada',
                                                             'DataDevolucao': 'data_devolucao',
                                                             'HoraDevolucao': 'hora_devolucao',
                                                             'EstacaoRetirada': 'estacao_retirada',
                                                             'EstacaoDevolucao': 'estacao_devolucao'});
# set id_jornada to index
clean_data_2016 = clean_data_2016.set_index('id_jornada')

# change type of data_cadastro
clean_data_2016['data_cadastro'] = clean_data_2016['data_cadastro'].astype('datetime64')

# combine data_retirada with hora_retirada
clean_data_2016['datetime_retirada'] = clean_data_2016['data_retirada'] + pd.to_timedelta(clean_data_2016['hora_retirada'])

# combine data_devolucao with hora_devolucao
clean_data_2016['datetime_devolucao'] = clean_data_2016['data_devolucao'] + pd.to_timedelta(clean_data_2016['hora_devolucao'])

# drop columns with duplicate information columns
clean_data_2016 = clean_data_2016.drop(['TempoJornada',
                                        'TempoJornadaMinutos',
                                        'data_retirada',
                                        'hora_retirada',
                                        'data_devolucao',
                                        'hora_devolucao',
                                        'distrito'], axis=1)

### 2017

In [4]:
# function to transform date inputs of '%d/%m/%Y' to '%Y-%m-%d'
import re

def my_func(row):
    mat=re.match('(\d{2})[/](\d{2})[/](\d{4})[ ](\d{2})[:](\d{2})[:](\d{2})$', str(row))
    if mat is not None:
        return datetime.datetime.strptime(row, "%d/%m/%Y %H:%M:%S").strftime('%Y')
    else:
        return datetime.datetime.strptime(str(row), "%Y-%m-%d %H:%M:%S").strftime('%Y')

In [6]:
# load the data
brute_path_2017 = '../data/brute/2017.xlsx'
brute_data_2017 = pd.read_excel(brute_path_2017)

# fix wrong date inputs
brute_data_2017['ano_nascimento'] = brute_data_2017.apply(lambda row: my_func(row['Nascimento']), axis=1)
brute_data_2017['ano_nascimento'].astype('int')

# change the columns name
clean_data_2017 = brute_data_2017.rename(index=str, columns={'IdJornada': 'id_jornada',
                                                             'globalId': 'id_usuario',
                                                             'Sexo': 'sexo',
                                                             'País': 'pais',
                                                             'Distrito': 'distrito',
                                                             'Cidade': 'cidade',
                                                             'UF': 'uf',
                                                             'Data de Cadastro': 'data_cadastro',
                                                             'Meio de Retirada': 'tipo_usuario',
                                                             'NumExterno': 'id_bicicleta',
                                                             'DataCorrida': 'data_corrida',
                                                             'HoraRetirada': 'hora_retirada',
                                                             'DataDevolucao': 'data_devolucao',
                                                             'HoraDevolucao': 'hora_devolucao',
                                                             'EstacaoRetirada': 'estacao_retirada',
                                                             'EstacaoDevolucao': 'estacao_devolucao'});

# # set index column
clean_data_2017 = clean_data_2017.set_index('id_jornada')

# merge data_corrida with hora_retirada
clean_data_2017['datetime_retirada'] = clean_data_2017['data_corrida'] + pd.to_timedelta(clean_data_2017['hora_retirada'].astype(str))

# merge data_corrida with hora_devolucao
clean_data_2017['datetime_devolucao'] = clean_data_2017['data_corrida'] + pd.to_timedelta(clean_data_2017['hora_devolucao'].astype(str))

# drop columns with duplicate information columns
clean_data_2017 = clean_data_2017.drop(['Nascimento',
                                        'Projeto',
                                        'DiaSemana',
                                        'AreaEstacaoRetirada',
                                        'EnderecoEstacaoRetirada',
                                        'AreaEstacaoDevolucao',
                                        'EnderecoEstacaoDevolucao',
                                        'Duração da Corrida',
                                        'data_corrida',
                                        'hora_retirada',
                                        'hora_devolucao'], axis=1)

In [16]:
clean_data_2017.columns.values

array(['id_usuario', 'sexo', 'pais', 'cidade', 'uf', 'data_cadastro',
       'estacao_retirada', 'tipo_usuario', 'estacao_devolucao',
       'id_bicicleta', 'ano_nascimento', 'datetime_retirada',
       'datetime_devolucao'], dtype=object)

In [14]:
clean_data_2016.columns.values

array(['id_usuario', 'ano_nascimento', 'sexo', 'pais', 'cidade', 'uf',
       'data_cadastro', 'tipo_usuario', 'id_bicicleta',
       'estacao_retirada', 'estacao_devolucao', 'datetime_retirada',
       'datetime_devolucao'], dtype=object)

In [12]:
clean_data_2015.columns

Index(['id_usuario', 'ano_nascimento', 'sexo', 'pais', 'cidade', 'uf',
       'data_cadastro', 'tipo_usuario', 'id_bicicleta', 'estacao_retirada',
       'estacao_devolucao', 'datetime_retirada', 'datetime_devolucao'],
      dtype='object')

In [17]:
clean_data_all = pd.concat([clean_data_2015, clean_data_2016, clean_data_2017])

/home/omadson/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [18]:
clean_data_all

,ano_nascimento,cidade,data_cadastro,datetime_devolucao,datetime_retirada,estacao_devolucao,estacao_retirada,id_bicicleta,id_usuario,pais,sexo,tipo_usuario,uf
id_jornada,,,,,,,,,,,,,
1655860,1989,NaN,2014-12-15 22:11:59.457000,2014-12-16 06:33:31.307,2014-12-16 06:12:43.517,14 - Aterro Praia de Iracema,04 - BNB Clube,10561,1644398,BR,M,URA/APP,CE
1655861,1985,NaN,2014-12-15 22:45:39.280000,2014-12-16 07:08:10.530,2014-12-16 06:58:04.743,01 - Praça Luíza Távora,01 - Praça Luíza Távora,10460,1644548,BR,M,URA/APP,CE
1655862,1995,NaN,2014-12-16 06:19:17.760000,2014-12-16 07:34:38.240,2014-12-16 06:58:32.983,15 - Naútico,15 - Naútico,10464,1644977,BR,F,URA/APP,CE
1655863,1987,fortaleza,2014-12-14 14:59:47.703000,2014-12-16 07:52:22.063,2014-12-16 07:01:22.367,14 - Aterro Praia de Iracema,01 - Praça Luíza Távora,10637,1639126,BR,M,URA/APP,CE
1655864,1980,NaN,2014-12-16 05:18:45.207000,2014-12-16 07:10:07.580,2014-12-16 07:07:44.813,08 - Joaquim Nabuco,08 - Joaquim Nabuco,10576,1644927,BR,M,URA/APP,CE
1655865,1980,NaN,2014-12-16 05:18:45.207000,2014-12-16 07:26:16.473,2014-12-16 07:11:10.390,08 - Joaquim Nabuco,08 - Joaquim Nabuco,10556,1644927,BR,M,URA/APP,CE
1655866,1989,NaN,2014-12-15 22:11:59.457000,2014-12-16 07:49:46.060,2014-12-16 07:30:44.947,04 - BNB Clube,14 - Aterro Praia de Iracema,10624,1644398,BR,M,URA/APP,CE
1655867,1984,NaN,2014-12-14 22:45:57.517000,2014-12-16 07:53:49.850,2014-12-16 07:31:40.237,15 - Naútico,15 - Naútico,10629,1640615,BR,M,URA/APP,CE
1655868,1989,Fortaleza,2012-12-04 06:17:21.147000,2014-12-16 08:14:43.000,2014-12-16 07:32:13.617,15 - Naútico,15 - Naútico,10583,400675,BR,M,URA/APP,CE
